In [3]:
import random
import string
import time

# 1. Przyjmij następujący zbiór danych wejściowych:

In [4]:
data1 = "bbb$"
data2 = "aabbabd"
data3 = "ababcd"
data4 = "abaababaabaabaabab$"
data5 = ''.join(random.choices(string.ascii_lowercase, k=100))
data6 = None

with open('1997_714_head.txt', 'r') as file:
    data6 = file.read()

# 2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

In [5]:
def marker_check(text, marker='$'):
    lines = text.splitlines()
    processed = []

    for line in lines:
        if not line.endswith(marker):
            line += marker
        processed.append(line)

    return '\n'.join(processed)


test_data = "abc"
print("Before: " + test_data)
processed_data = marker_check(test_data)
print("After: " + processed_data)
print("-------------------")

test_data = "abcd$"
print("Before: " + test_data)
processed_data = marker_check(test_data)
print("After: " + processed_data)
print("-------------------")


Before: abc
After: abc$
-------------------
Before: abcd$
After: abcd$
-------------------


# 3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [14]:
class TrieNode:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = {}

class Trie:
    def __init__(self, text):
        self.root = TrieNode()
        self.build(text)

    def build(self, text):
        n = len(text)
        for i in range(n):
            suffix = text[i:]
            node = self.root
            for char in suffix:
                # jeśli nie ma takiej literki to trzeba stworzyć nowy node
                if char not in node.children:
                    new = TrieNode(node)
                    node.children[char] = new
                node = node.children[char]

    def search(self, pattern):
        node = self.root
        for char in pattern:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

# 4. Zaimplementuj algorytm konstruujący drzewo sufiksów.

In [15]:
#TODO

# 5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.

Algorytm wyszukiwania wzorców (Algorytm Knutha-Morrisa-Pratta) został wzięty z laboratorium pierwszego.

In [16]:
def prefix_function(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0

    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k

    return pi


def KMP(text, pattern):
    n = len(text)
    m = len(pattern)
    pi = prefix_function(pattern)
    q = 0
    result = []

    for i in range(n):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == m:
            result.append(i - m + 1)
            q = pi[q-1]

    return result

Funkcja testująca:

In [17]:
def test_(text, pattern):
    trie = Trie(text)
    trie_occurrence = trie.search(pattern)
    occ = KMP(text, pattern)
    kmp_occurrence = len(occ) > 0
    if kmp_occurrence and trie_occurrence:
        print("Znaleziono wzorzec.")
    else:
        print("Nie znaleziono wzorca.")

        
test_("bananasbananas", "ananas")
test_("aaaaaaaaaa", "a")
test_("bananasbananas", "banana")
test_("asd","sd")

Znaleziono wzorzec.
Znaleziono wzorzec.
Znaleziono wzorzec.
Znaleziono wzorzec.


# 6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
<ul>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (1p.),</li>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (1p.),</li>
    <li>Drzewo sufiksów - algorytm Ukkonena (3p).</li>
    </ul>

In [18]:
# ad1: Trie - w wariancie, w którym kolejne sufiksy dodawane
# są przez przeszukiwanie głowy od korzenia drzewa

def measure_time(data):
    start_time = time.time()
    trie = Trie(data)
    time_trie = time.time() - start_time
    print(f"Czas konstrukcji Trie: {time_trie} s")
    
print("Tests for data1")
print("----------------------")
measure_time(data1)
print("")
print("Tests for data2")
print("----------------------")
measure_time(data2)
print("")
print("Tests for data3")
print("----------------------")
measure_time(data3)
print("")
print("Tests for data4")
print("----------------------")
measure_time(data4)
print("")
print("Tests for data5")
print("----------------------")
measure_time(data5)
print("")
print("Tests for data6")
print("----------------------")
measure_time(data6)

Tests for data1
----------------------
Czas konstrukcji Trie: 1.811981201171875e-05 s

Tests for data2
----------------------
Czas konstrukcji Trie: 2.9325485229492188e-05 s

Tests for data3
----------------------
Czas konstrukcji Trie: 2.4080276489257812e-05 s

Tests for data4
----------------------
Czas konstrukcji Trie: 0.00028705596923828125 s

Tests for data5
----------------------
Czas konstrukcji Trie: 0.007759809494018555 s

Tests for data6
----------------------
Czas konstrukcji Trie: 16.79684615135193 s
